### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / IFSC Climbing European Championships (International Federation of Sport Climbing)

The IFSC Climbing European Championships are the biennial European championship for competition climbing organized by the  IFSC. The first competition was held in Frankfurt in 1992

This event determines the male and female european champions in the 4 disciplines of competition climbing: 

competition LEAD climbing, competition BOULDERING, competition SPEED CLIMBING, Since 2015-2020, competition COMBINED CLIMBING. 

In [1]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_Staging'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [2]:
### url
Wiki_IFSC_Climbing_European_Championships = "https://en.wikipedia.org/wiki/IFSC_Climbing_European_Championships"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_IFSC_Climbing_European_Championships = pd.read_html(Wiki_IFSC_Climbing_European_Championships)
type(Dataset_Wiki_IFSC_Climbing_European_Championships), len(Dataset_Wiki_IFSC_Climbing_European_Championships)

(list, 29)

##### on récupère 1 list de 29 élts

In [136]:
type(Dataset_Wiki_IFSC_Climbing_European_Championships[0])

pandas.core.frame.DataFrame

##### > Chaque élt est un DataFrame Pandas
##### ? Quel est le contenu chacun des 29 dataframes ? Lesquelles nous interessent ?

In [137]:
Dataset_Wiki_IFSC_Climbing_European_Championships[0]

,European Climbing Championships,European Climbing Championships.1
0,Status,active
1,Genre,sporting event
2,Date(s),mid-year
3,Frequency,biennial
4,Country,varying
5,Inaugurated,1992


######## [0] is dataset of summarized data about Climbing European championships dataset > NOT to Discard

In [138]:
Dataset_Wiki_IFSC_Climbing_European_Championships[1]

Number   Year             Location                  Date(s) Disciplines  \
   Number   Year             Location                  Date(s)        lead   
0       1   1992            Frankfurt             September 18           •   
1       2   1996                Paris               January 28           •   
2       3   1998            Nuremberg                  April 3           •   
3       4   2000               Munich                October 7           •   
4       5   2002             Chamonix               July 11–14           •   
5       6   2004                Lecco               June 21–27           •   
6       7   2006        Yekaterinburg         June 29 – July 3           •   
7       7  20071           Birmingham              March 16–18         NaN   
8       8   2008                Paris            October 15–18           •   
9       9   2010       Imst/Innsbruck          September 14–18           •   
10     10   2013             Chamonix               July 12–13           •   
11     10   2013            Eindhoven  August 31 – September 1         NaN   
12     11   2015            Innsbruck                May 14–16         NaN   
13     11   2015             Chamonix               July 10–12           •   
14     12   2017  Campitello di Fassa         June 30 – July 1           •   
15     12   2017               Munich             August 18–19         NaN   
16     13   2019             Zakopane            September 5–7         NaN   
17     13   2019            Edinburgh              October 4–6           •   
18     14   2020               Moscow           November 21–28           •   
19     15   2022               Munich             August 11–21           •   
20     16   2024    Villars-sur-Ollon  August 27 – September 1           •   

                            Notes  
   speed   boulder combined Notes  
0      •       NaN      NaN   [2]  
1      •       NaN      NaN   [3]  
2      •       NaN      NaN   [4]  
3      •       NaN      NaN   [5]  
4      •         •      NaN   [6]  
5      •         •      NaN   [7]  
6      •  canceled      NaN   [8]  
7    NaN         •      NaN   [9]  
8      •         •      NaN  [10]  
9      •         •      NaN  [11]  
10     •       NaN      NaN  [12]  
11   NaN         •      NaN  [13]  
12   NaN         •      NaN  [14]  
13     •       NaN      NaN  [15]  
14     •       NaN      NaN  [16]  
15   NaN         •      NaN  [17]  
16   NaN         •      NaN  [18]  
17     •       NaN      NaN  [18]  
18     •         •        •  [18]  
19     •         •        •   NaN  
20     •         •        •  [19]

######## [1] is the List of Climbing European Championships

In [139]:
Dataset_Wiki_IFSC_Climbing_European_Championships[1].columns

MultiIndex([(     'Number',   'Number'),
            (       'Year',     'Year'),
            (   'Location', 'Location'),
            (    'Date(s)',  'Date(s)'),
            ('Disciplines',     'lead'),
            ('Disciplines',    'speed'),
            ('Disciplines',  'boulder'),
            ('Disciplines', 'combined'),
            (      'Notes',    'Notes')],
           )

In [140]:
# Delete Multi Index
# Replace Date par Period (car SQL à avoir format Date, non applicable ici)
Dataset_Wiki_IFSC_Climbing_European_Championships[1].columns = ['Number','Year','Location','Period','LEAD','SPEED','BOULDER','COMBINED','Notes']
Dataset_Wiki_IFSC_Climbing_European_Championships[1]

,Number,Year,Location,Period,LEAD,SPEED,BOULDER,COMBINED,Notes
0,1,1992,Frankfurt,September 18,•,•,NaN,NaN,[2]
1,2,1996,Paris,January 28,•,•,NaN,NaN,[3]
2,3,1998,Nuremberg,April 3,•,•,NaN,NaN,[4]
3,4,2000,Munich,October 7,•,•,NaN,NaN,[5]
4,5,2002,Chamonix,July 11–14,•,•,•,NaN,[6]
5,6,2004,Lecco,June 21–27,•,•,•,NaN,[7]
6,7,2006,Yekaterinburg,June 29 – July 3,•,•,canceled,NaN,[8]
7,7,20071,Birmingham,March 16–18,NaN,NaN,•,NaN,[9]
8,8,2008,Paris,October 15–18,•,•,•,NaN,[10]
9,9,2010,Imst/Innsbruck,September 14–18,•,•,•,NaN,[11]


!! ATTENTION erreur de Date ligne 7 20071 au lieu de 2007

In [191]:
Dataset_Wiki_IFSC_Climbing_European_Championships[1].dtypes

Number       int64
Year         int64
Location    object
Period      object
LEAD        object
SPEED       object
BOULDER     object
COMBINED    object
Notes       object
dtype: object

In [192]:
Wiki_IFSC_Climbing_European_Championships_Edition_List_Df1 = Dataset_Wiki_IFSC_Climbing_European_Championships[1]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Edition_List_Df1
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Number INT,
                                        Year INT,
                                        Location VARCHAR(100),
                                        Period VARCHAR(50),
                                        LEAD VARCHAR(50),
                                        SPEED VARCHAR(50),
                                        BOULDER VARCHAR(50),
                                        COMBINED VARCHAR(50),
                                        Notes VARCHAR(50)                                                           
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Edition_List_Df1.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Edition_List_Df1',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

err: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207)")


4 Disciplines : From 1992 : Lead and Speed. from 2002 : Boulder(ing). from 2020 : Combined

In [142]:
Dataset_Wiki_IFSC_Climbing_European_Championships[2]

,Year,Gold,Silver,Bronze
0,1992,François Legrand,François Petit,Salavat Rakhmetov
1,1996,Arnaud Petit,François Petit,François Lombard
2,1998,Ian Vickers,Cristian Brenna,Christian Bindhammer
3,2000,Alexandre Chabot,Cristian Brenna,Evgeny Ovchinnikov
4,2002,Alexandre Chabot,Tomáš Mrázek,Ramón Julián Puigblanqué
5,2004,Ramón Julián Puigblanqué,Alexandre Chabot,François Auclair
6,2006,David Lama,Cédric Lachat,Dmitri Sarafutdinov
7,2008,Patxi Usobiaga Lakunza,Tomáš Mrázek,Manuel Romain
8,2010,Ramón Julián Puigblanqué,Adam Ondra,Jakob Schubert
9,2013,Romain Desgranges,Ramón Julián Puigblanque,Jorg Verhoeven


[2] is about Men Medal winner full names of Climbing European Championships in Disciplin LEAD

In [181]:
Wiki_IFSC_Climbing_European_Championships_Medal_Men_Lead_Df2 = Dataset_Wiki_IFSC_Climbing_European_Championships[2]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Men_Lead_Df2
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Men_Lead_Df2.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Men_Lead_Df2',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [144]:
Dataset_Wiki_IFSC_Climbing_European_Championships[3]

,Year,Gold,Silver,Bronze
0,2002,Christian Core,Mauro Calibani,Salavat Rakhmetov
1,2004,Daniel Dulac,Andrew Earl,Gabriele Moroni
2,2007,David Lama,Nalle Hukkataival,Tomáš Mrázek
3,2008,Jérôme Meyer,Kilian Fischhuber,Cédric Lachat
4,2010,Cédric Lachat,Adam Ondra,Kilian Fischhuber
5,2013,Kilian Fischhuber,Dmitri Sarafutdinov,Jakob Schubert
6,2015,Jan Hojer,Adam Ondra,Stefan Scarperi
7,2017,Jan Hojer,Alexander Megos,Anže Peharc
8,2019,Mickaël Mawem,Sergei Skorodumov,Vadim Timonov
9,2020,Jernej Kruder,Sergei Luzhetskii,Nikolai Iarilovets


[3] is about Men Medal winner full names of Climbing European Championships in Disciplin BOULDERING

In [182]:
Wiki_IFSC_Climbing_European_Championships_Medal_Men_Bouldering_Df3 = Dataset_Wiki_IFSC_Climbing_European_Championships[3]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Men_Bouldering_Df3
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Men_Bouldering_Df3.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Men_Bouldering_Df3',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [146]:
Dataset_Wiki_IFSC_Climbing_European_Championships[4]

,Year,Gold,Silver,Bronze
0,1992,Kairat Rakhmetov,Cristian Brenna,Andrzej Marcisz
1,1996,Andrey Vedenmeer,Mathieu Dutray,Pavel Samoiline
2,1998,Tomasz Oleksy,Alexander Paukaev,Vladimir Zakharov
3,2000,Maksym Styenkovyy,Iakov Soubbotine,Alexander Chaoulsky
4,2002,Maksym Styenkovyy,Sergey Sinitcyn,Alexei Gadeev
5,2004,Alexander Peshekhonov,Maksym Styenkovyy,Alexander Chaoulsky
6,2006,Evgeny Vaitcekhovsky,Sergey Sinitcyn,Alexander Peshekhonov
7,2008,Evgeny Vaitcekhovsky,Maksym Osipov,Maksym Styenkovyy
8,2010,Sergey Abdrakhmanov,Stanislav Kokorin,Libor Hroza
9,2013,Libor Hroza,Marcin Dzieński,Danyil Boldyrev


[4] is about Men Medal winner full names of Climbing European Championships in Disciplin SPEED

In [183]:
Wiki_IFSC_Climbing_European_Championships_Medal_Men_Speed_Df4 = Dataset_Wiki_IFSC_Climbing_European_Championships[4]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Men_Speed_Df4
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Men_Speed_Df4.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Men_Speed_Df4',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [148]:
Dataset_Wiki_IFSC_Climbing_European_Championships[5]

,Year,Gold,Silver,Bronze
0,2015,Jakob Schubert,Stefano Ghisolfi,Javier Cano Blázquez
1,2017,Jan Hojer,Jakob Schubert,Michael Piccolruaz
2,2020,Alexey Rubtsov,Sascha Lehmann,Sergey Luzhetsky
3,2022,Jakob Schubert,Adam Ondra,Alberto Ginés López


[5] is about Men Medal winner full names of Climbing European Championships in Disciplin COMBINED (Overall)

In [184]:
Wiki_IFSC_Climbing_European_Championships_Medal_Men_Combined_Df5 = Dataset_Wiki_IFSC_Climbing_European_Championships[5]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Men_Combined_Df5
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Men_Combined_Df5.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Men_Combined_Df5',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [150]:
Dataset_Wiki_IFSC_Climbing_European_Championships[6]

,Year,Gold,Silver,Bronze
0,1992,Susi Good,Isabelle Patissier,Laurence Guyon
1,1996,Liv Sansoz,Laurence Guyon,Stéphanie Bodet Venera Chereshneva
2,1998,Muriel Sarkany,Venera Chereshneva,Liv Sansoz
3,2000,Katrin Sedlmayer,Stéphanie Bodet,Marietta Uhden
4,2002,Chloé Minoret,Martina Cufar,Sandrine Levet Muriel Sarkany
5,2004,Bettina Schöpf,Natalija Gros,Katharina Saurwein
6,2006,Charlotte Durif,Sandrine Levet,Maja Vidmar
7,2008,Johanna Ernst,Maja Vidmar,Mina Markovič
8,2010,Angela Eiter,Johanna Ernst,Alizée Dufraisse
9,2013,Dinara Fakhritdinova,Mina Markovič,Hélène Janicot


[6] is about Women Medal winner full names of Climbing European Championships in Disciplin LEAD

In [185]:
Wiki_IFSC_Climbing_European_Championships_Medal_Women_Lead_Df6 = Dataset_Wiki_IFSC_Climbing_European_Championships[6]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Women_Lead_Df6
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Women_Lead_Df6.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Women_Lead_Df6',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [152]:
Dataset_Wiki_IFSC_Climbing_European_Championships[7]

,Year,Gold,Silver,Bronze
0,2002,Sandrine Levet,Fanny Rogeaux,Vera Kotasova-Kostruhova
1,2004,Olga Bibik,Anna Stöhr,Corinne Theroux
2,2007,Juliette Danion,Olga Shalagina,Olga Bezhko
3,2008,Natalija Gros,Anna Stöhr,Hannah Midtboe
4,2010,Anna Stöhr,Juliane Wurm,Olga Shalagina
5,2013,Anna Stöhr,Mina Markovič,Mélanie Sandoz
6,2015,Juliane Wurm,Anna Stöhr,Katharina Saurwein
7,2017,Staša Gejo,Janja Garnbret,Petra Klingler
8,2019,Urska Repusic,Vita Lukan,Irina Kuzmenko
9,2020,Viktoriia Meshkova,Chloe Caulier,Staša Gejo


[7] is about Women Medal winner full names of Climbing European Championships in Disciplin BOULDERING

In [186]:
Wiki_IFSC_Climbing_European_Championships_Medal_Women_Bouldering_Df7 = Dataset_Wiki_IFSC_Climbing_European_Championships[7]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Women_Bouldering_Df7
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Women_Bouldering_Df7.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Women_Bouldering_Df7',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [154]:
Dataset_Wiki_IFSC_Climbing_European_Championships[8]

,Year,Gold,Silver,Bronze
0,1992,Yelena Ovchinnikova,Yulia Inozemtseva,Claudine Trecourt
1,1996,Cecile Avezou,Ameli Haager,Kim Anthoni
2,1998,Cecile Avezou,Olena Ryepko,Zosia Podgorbounskikh
3,2000,Olena Ryepko,Tatiana Ruyga,Zosia Podgorbounskikh
4,2002,Olena Ryepko,Olga Zakharova,Svetlana Sutkina
5,2004,Anna Stenkovaya,Valentina Yurina,Maya Piratinskaya
6,2006,Anna Stenkovaya,Kseniia Alekseeva,Valentina Yurina
7,2008,Edyta Ropek,Olga Morozkina,Anna Stenkovaya
8,2010,Edyta Ropek,Kseniia Alekseeva,Natalia Titova
9,2013,Anna Tsyganova,Aleksandra Rudzińska,Yulia Levochkina


[8] is about Women Medal winner full names of Climbing European Championships > NOT to Discard

In [187]:
Wiki_IFSC_Climbing_European_Championships_Medal_Women_Speed_Df8 = Dataset_Wiki_IFSC_Climbing_European_Championships[8]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Women_Speed_Df8
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Women_Speed_Df8.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Women_Speed_Df8',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [156]:
Dataset_Wiki_IFSC_Climbing_European_Championships[9]

,Year,Gold,Silver,Bronze
0,2015,Jessica Pilz,Mina Markovič,Charlotte Durif
1,2017,Janja Garnbret,Petra Klingler,Staša Gejo
2,2020,Viktoria Meshkova,Staša Gejo,Eliska Adamovska
3,2022,Janja Garnbret,Mia Krampl,Jessica Pilz


[9] is about Women Medal winner full names of Climbing European Championships in Disciplin COMBINED

In [190]:
Wiki_IFSC_Climbing_European_Championships_Medal_Women_Combined_Df9 = Dataset_Wiki_IFSC_Climbing_European_Championships[9]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Medal_Women_Combined_Df9
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year INT,
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Medal_Women_Combined_Df9.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Medal_Women_Combined_Df9',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

#### Pour les 19  (de #10 à #28 ) suivants de la liste de 29 DataFrames

In [158]:
# Test pour afficher d' 1 coup l'ensemble des 19 tables suivantes (de #10 à #28 ) de la liste de 29 DataFrames
""" for i in range (10,len(Dataset_Wiki_IFSC_Climbing_European_Championships),1): # boucle i de 10 inclus à 28 = (29-1)
    Dataset_Wiki_IFSC_Climbing_European_Championships[i]
    if len(Dataset_Wiki_IFSC_Climbing_European_Championships) > 50: 
         break"""
# Run est arrêté manuellement car dure  > ~10min

' for i in range (10,len(Dataset_Wiki_IFSC_Climbing_European_Championships),1): # boucle i de 10 inclus à 28 = (29-1)\n    Dataset_Wiki_IFSC_Climbing_European_Championships[i]\n    if len(Dataset_Wiki_IFSC_Climbing_European_Championships) > 50: \n         break'

In [3]:
Dataset_Wiki_IFSC_Climbing_European_Championships[10]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,Russia,17,19,28,64
1,2,France,17,10,18,45
2,3,Austria,12,7,9,28
3,4,Slovenia,9,12,3,24
4,5,Ukraine,7,9,5,21
5,6,Poland,6,4,4,14
6,7,Germany,5,4,3,12
7,8,Czech Republic,4,9,5,18
8,9,Spain,4,2,4,10
9,10,Switzerland,3,3,3,9


[10]  is about European Countries' Ranking (M/W mixed = 'Total') in Climbing European Championships (based on medals points)

In [5]:
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[0,1] = 'Fédération de Russie'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[1,1] = 'France'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[2,1] = 'Autriche'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[3,1] = 'Slovénie'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[4,1] = 'Ukraine'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[5,1] = 'Pologne'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[6,1] = 'Allemagne'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[7,1] = 'République tchèque'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[8,1] = 'Espagne'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[9,1] = 'Suisse'
Dataset_Wiki_IFSC_Climbing_European_Championships[10].iloc[10,1] = 'Belgique'



In [6]:
Dataset_Wiki_IFSC_Climbing_European_Championships[10]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,Fédération de Russie,17,19,28,64
1,2,France,17,10,18,45
2,3,Autriche,12,7,9,28
3,4,Slovénie,9,12,3,24
4,5,Ukraine,7,9,5,21
5,6,Pologne,6,4,4,14
6,7,Allemagne,5,4,3,12
7,8,République tchèque,4,9,5,18
8,9,Espagne,4,2,4,10
9,10,Suisse,3,3,3,9


In [195]:
Dataset_Wiki_IFSC_Climbing_European_Championships[10].dtypes

Rank      object
Nation    object
Gold       int64
Silver     int64
Bronze     int64
Total      int64
dtype: object

In [11]:
Wiki_IFSC_Climbing_European_Championships_Countries_Total_Ranking_Df10 = Dataset_Wiki_IFSC_Climbing_European_Championships[10].head(10)

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_European_Championships_Countries_Total_Ranking_Df10
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rank VARCHAR(50),
                                        Nation VARCHAR(50),
                                        Gold INT,
                                        Silver INT,
                                        Bronze INT,
                                        Total INT                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_European_Championships_Countries_Total_Ranking_Df10.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_European_Championships_Countries_Total_Ranking_Df10',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [161]:
Dataset_Wiki_IFSC_Climbing_European_Championships[11]

,vteEuropean Championships,vteEuropean Championships.1
0,Olympic sports,Team sports Association football Men Women Men...
1,Team sports,Association football Men Women Men's club Wome...
2,Individual sports,Archery Athletics Badminton Biathlon Bobsleigh...
3,Non-Olympic sports,Team sports American football Australian rules...
4,Team sports,American football Australian rules football Be...
5,Individual sports,Athletics Cross-country Indoor Mountain runnin...
6,Paralympic sports,Parasports Athletics Badminton CP Football Jud...
7,Parasports,Athletics Badminton CP Football Judo Para arch...
8,Motor sports,Cars Formula 3 Hillclimbing Karting KF1 Radio-...
9,Cars,Formula 3 Hillclimbing Karting KF1 Radio-contr...


[11] is NOT about Climbing European Championships Climbing : Sport in general, Only pertinent info is Climbling is an Olympic sports. > to discard !!

In [162]:
Dataset_Wiki_IFSC_Climbing_European_Championships[12]

,0,1
0,Team sports,Association football Men Women Men's club Wome...
1,Individual sports,Archery Athletics Badminton Biathlon Bobsleigh...


[12] is NOT about Climbing European Championships Climbing : Sport in general, not Climbing specially :  > to discard !!

In [163]:
Dataset_Wiki_IFSC_Climbing_European_Championships[13]

,0,1
0,Team sports,American football Australian rules football Be...
1,Individual sports,Athletics Cross-country Indoor Mountain runnin...


[13] is NOT about Climbing European Championships Climbing : Sport in general, not Climbing specially  :  > to discard !!

In [164]:
Dataset_Wiki_IFSC_Climbing_European_Championships[14]

,0,1
0,Parasports,Athletics Badminton CP Football Judo Para arch...


[14] is NOT about Climbing European Championships Climbing : Sport in general, not Climbing specially  :  > to discard !!

In [165]:
Dataset_Wiki_IFSC_Climbing_European_Championships[15]

,0,1
0,Cars,Formula 3 Hillclimbing Karting KF1 Radio-contr...
1,Motorcycles,Speedway Individual Pairs Club Ice Team Motocr...
2,Powered aviation,Aerobatics Air racing


[15] is NOT about Climbing European Championships Climbing : Sport in general, not Climbing specially : (cars, aviation) > to discard !!

In [166]:
Dataset_Wiki_IFSC_Climbing_European_Championships[16]

,vteClimbing competitions,vteClimbing competitions.1
0,Disciplines,Lead climbing (also called sport climbing) Bou...
1,Rankings,Ranking of career IFSC victories by climber
2,Rock,WorldIFSC IFSC Climbing World Cup IFSC Climbin...
3,World,IFSC IFSC Climbing World Cup IFSC Climbing Wor...
4,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
5,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...
6,Continental,IFSC African Championships African Youth Champ...
7,IFSC,African Championships African Youth Championsh...
8,Multi-sport,Asian Games European Games
9,National,American Championships Austrian Championships ...


[16] is about other Climbing Championships (nor European neither WorldWide Championships) or Multi-Sport Championships : [1] List of Disciplines, [1] Rankings of career IFSC victories by climber, [2-3-4] 'Rock Climbing', [5] Multi-Sport Summer , [6-7] Continental, [8] Multi-Sport, [9] National, [10] Individual, [11-12] 'Ice Climbing' , [13-14] International Organisations of 'Sport Climbing', [15] USA Climbing. Some Lines are Interesting for the Data Analysis Project > NOT to discard

In [167]:
Dataset_Wiki_IFSC_Climbing_European_Championships[17]

,0,1
0,World,IFSC IFSC Climbing World Cup IFSC Climbing Wor...
1,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
2,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...
3,Continental,IFSC African Championships African Youth Champ...
4,IFSC,African Championships African Youth Championsh...
5,Multi-sport,Asian Games European Games
6,National,American Championships Austrian Championships ...
7,Individual,Sportroccia Rock Master Melloblocco Nordic Cha...


[17] is duplicate of [16]: to discard

In [168]:
Dataset_Wiki_IFSC_Climbing_European_Championships[18]

,0,1
0,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
1,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...


[18] is duplicate of [16-17]: to discard

In [169]:
Dataset_Wiki_IFSC_Climbing_European_Championships[19]

,0,1
0,IFSC,African Championships African Youth Championsh...
1,Multi-sport,Asian Games European Games


[19] is duplicate of [16-17-18]: to discard

In [170]:
Dataset_Wiki_IFSC_Climbing_European_Championships[20]

,0,1
0,World,UIAA Ice Climbing World Cup UIAA Ice Climbing ...


[20] is duplicate of [16]: to discard

In [171]:
Dataset_Wiki_IFSC_Climbing_European_Championships[21]

,0,1
0,International,International Federation of Sport Climbing (IF...
1,Other,USA Climbing


[21] is duplicate of [16]: to discard

In [172]:
Dataset_Wiki_IFSC_Climbing_European_Championships[22]

,vteClimbing,vteClimbing.1
0,Types,Rock climbingDisciplines Aid Big wall Multi-pi...
1,Rock climbing,Disciplines Aid Big wall Multi-pitch Boulderin...
2,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
3,Rock types,Crack Face Roof Slab
4,Mountaineering,Alpine Via ferrata Himalayan Alpine style Expe...
5,Other,Buildering Canyoning Crane Grass Commercial Pa...
6,Lists,Alpine clubs Climbers Piolet d'Or winners IFSC...
7,Terminology,EquipmentRock Aider Daisy chain Anchor Ascende...
8,Equipment,Rock Aider Daisy chain Anchor Ascender Croll B...
9,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...


In [173]:
Dataset_Wiki_IFSC_Climbing_European_Championships[23]

,0,1
0,Rock climbing,Disciplines Aid Big wall Multi-pitch Boulderin...
1,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
2,Rock types,Crack Face Roof Slab
3,Mountaineering,Alpine Via ferrata Himalayan Alpine style Expe...
4,Other,Buildering Canyoning Crane Grass Commercial Pa...


[23] is duplicate of [22] > To discard

In [174]:
Dataset_Wiki_IFSC_Climbing_European_Championships[24]

,0,1
0,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
1,Rock types,Crack Face Roof Slab


[24] is duplicate of [23] > To discard

In [175]:
Dataset_Wiki_IFSC_Climbing_European_Championships[25]

,0,1
0,Equipment,Rock Aider Daisy chain Anchor Ascender Croll B...
1,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...
2,Ice and snow,Alpenstock Abalakov thread Bottled oxygen Cram...
3,Action,Abseiling Australian Dülfersitz Belaying Front...
4,Other,Ape index Approach shoe Clean Climbing route B...


[25] is duplicate of  [22] > to Discard

In [176]:
Dataset_Wiki_IFSC_Climbing_European_Championships[26]

,0,1
0,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...
1,Ice and snow,Alpenstock Abalakov thread Bottled oxygen Cram...


[26] is duplicate of [22] > to Discard

In [177]:
Dataset_Wiki_IFSC_Climbing_European_Championships[27]

,0,1
0,Magazines and journals,Alpine Journal Alpinist American Alpine Journa...
1,Non-fiction films,The Alpinist El Capitan The Dawn Wall Free Sol...


[27] is duplicate of [22] > to Discard

In [178]:
Dataset_Wiki_IFSC_Climbing_European_Championships[28]

,0,1
0,National,Access Fund Alpine Club Alpine Club of Canada ...
1,International,International Federation of Sport Climbing (IF...


[28] is duplicate of [22] > to Discard